In [2]:
import easyocr
import torch
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torchmetrics.text import CharErrorRate
import cv2

%matplotlib inline

is_cuda = torch.cuda.is_available()

# init reader object
reader = easyocr.Reader(['en'], gpu=is_cuda, verbose=False)

In [3]:
data = os.listdir('ser_dataset')
ls_dir = []

for i in data:
    ls_dir.append(os.path.join('char_data', i))

ls_dir

['char_data/1IC-6855.jpg',
 'char_data/2BT-2188.jpg',
 'char_data/I-8676.jpg',
 'char_data/1HQ-6618.jpg',
 'char_data/SSH-8189.jpg',
 'char_data/2BQ-0556.jpg',
 'char_data/1GJ-94.jpg',
 'char_data/1J-9569.jpg',
 'char_data/1AR-1597.jpg',
 'char_data/1V-1069.jpg',
 'char_data/1FI-8479.jpg',
 'char_data/2AN-1687.jpg',
 'char_data/2A-7814.jpg',
 'char_data/1IJ-2469.jpg',
 'char_data/1A-7150.jpg',
 'char_data/1-8098.jpg',
 'char_data/2AN-0705.jpg',
 'char_data/2-9139.jpg',
 'char_data/1DE-1616.jpg',
 'char_data/2-1127.jpg',
 'char_data/1W-4780.jpg',
 'char_data/2BJ-6233.jpg',
 'char_data/1Q-4722.jpg',
 'char_data/1HB-2850.jpg',
 'char_data/1BB-8451.jpg',
 'char_data/1I-194.jpg',
 'char_data/1N-8833.jpg',
 'char_data/2BH-6221.jpg',
 'char_data/1FD-0345.jpg',
 'char_data/1FB-0107.jpg',
 'char_data/2BS-3495.jpg',
 'char_data/F.jpg',
 'char_data/EANG.SIM.jpg',
 'char_data/1CR-7542.jpg',
 'char_data/1T-7199.jpg',
 'char_data/1R-5644.jpg',
 'char_data/1JC-5801.jpg',
 'char_data/1M-3136.jpg',
 'c

In [ ]:
chars = []
for i in ls_dir:
    img = np.asarray(Image.open(i))
    plt.imshow(img)
    plt.show()
    chars.append(input('Input nuber: '))
    plt.clf()

In [4]:
import re

# removing spaces and other special characters
def remove_sp(string):
    result = re.sub(r'[^a-zA-Z0-9]', '', string)
    return result

In [5]:
non_sp_char = []

for ch in chars:
    non_sp_char.append(remove_sp(ch))

non_sp_char

NameError: name 'chars' is not defined

In [6]:
def pre_process(img):
    # Stretch the image to have a width of 640 and maintain the aspect ratio
    desired_width = 640
    aspect_ratio = img.shape[1] / img.shape[0]
    desired_height = int(desired_width / aspect_ratio)

    # Resize the image
    resized_img = cv2.resize(img, (desired_width, desired_height), interpolation=cv2.INTER_LINEAR)

    # Convert the image to grayscale
    gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 0)

    # Apply Otsu's thresholding
    _, thresholded_image = cv2.threshold(blurred_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # add sharpen filter
    sharpen_filter = np.array(
        [
            [-1,-1,-1],
            [-1, 9, -1],
            [-1,-1,-1]
        ]
    )

    sh_img = cv2.filter2D(thresholded_image, -1, sharpen_filter)

    return sh_img

In [11]:
import pandas as pd

data_dic = {
    'img_dir': ls_dir,
    'label': labels,
}

df = pd.DataFrame(data_dic)
df

,img_dir,label
0,char_data/1IC-6855.jpg,1IC-6855
1,char_data/2BT-2188.jpg,2BT-2188
2,char_data/I-8676.jpg,I-8676
3,char_data/1HQ-6618.jpg,1HQ-6618
4,char_data/SSH-8189.jpg,SSH-8189
...,...,...
865,char_data/1IK-5759.jpg,1IK-5759
866,char_data/1FC-7356.jpg,1FC-7356
867,char_data/1J-6032.jpg,1J-6032
868,char_data/1JC-4707.jpg,1JC-4707


[ WARN:0@746.749] global /croot/opencv-suite_1676452025216/work/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('char_data/1IC-6855.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [77]:
for i in range(len(df.predict)):
    df.clean_pre[i] = remove_sp(df.predict[i])

df

,img_dir,chars,n_sp_char,predict,clean_pre
0,char_data/16_77271.jpg,1M - 8209,1M8209,1M:8209,1M8209
1,char_data/3664_21201.jpg,4 - 0057,40057,:0U5,0U5
2,char_data/4459_33294.jpg,3A - 2450,3A2450,3A.2450,3A2450
3,char_data/1534_112238.jpg,1T - 6269,1T6269,II 5269,II5269
4,char_data/2736_55256.jpg,2B - 3172,2B3172,2B-3172,2B3172
5,char_data/4486_71231.jpg,2 - 0035,20035,"3 V 1 2,0035 RCd",3V120035RCd
6,char_data/3437_243278.jpg,BU - 8951,BU8951,2BU-8951,2BU8951
7,char_data/1947_35189.jpg,3 - 0033,30033,J-003?,J003
8,char_data/3503_70280.jpg,1J - 8626,1J8626,17.8626,178626
9,char_data/505_282292.jpg,2C - 0206,2C0206,2c-0206,2c0206


$$
    CER = \frac{S + D + I}{N_w}
$$

In [78]:
cer = CharErrorRate()

error = cer(df.chars, df.predict)

error

tensor(0.6190)

In [79]:
cl_er = cer(df.n_sp_char, df.clean_pre)

cl_er

tensor(0.2764)

In [80]:
# save file

df.to_csv('char_data.csv', index=False)

In [81]:
dataframe = pd.read_csv('char_data.csv')

dataframe

,img_dir,chars,n_sp_char,predict,clean_pre
0,char_data/16_77271.jpg,1M - 8209,1M8209,1M:8209,1M8209
1,char_data/3664_21201.jpg,4 - 0057,40057,:0U5,0U5
2,char_data/4459_33294.jpg,3A - 2450,3A2450,3A.2450,3A2450
3,char_data/1534_112238.jpg,1T - 6269,1T6269,II 5269,II5269
4,char_data/2736_55256.jpg,2B - 3172,2B3172,2B-3172,2B3172
5,char_data/4486_71231.jpg,2 - 0035,20035,"3 V 1 2,0035 RCd",3V120035RCd
6,char_data/3437_243278.jpg,BU - 8951,BU8951,2BU-8951,2BU8951
7,char_data/1947_35189.jpg,3 - 0033,30033,J-003?,J003
8,char_data/3503_70280.jpg,1J - 8626,1J8626,17.8626,178626
9,char_data/505_282292.jpg,2C - 0206,2C0206,2c-0206,2c0206


In [82]:
from jiwer import cer

er = []

for i in range(len(dataframe)):
    er.append(cer(dataframe.chars[i], dataframe.predict[i]))

np.mean(er)

0.5215277777777778

In [83]:
er = []

for i in range(len(dataframe)):
    er.append(cer(dataframe.n_sp_char[i], dataframe.clean_pre[i]))

np.mean(er)

0.30166666666666664